In [2]:
# load data
data = readRDS('../../ABCA7lof//processed_data/for_plotting/lipidomics.rds')
key = read.csv('../../ABCA7lof//raw_data/ngn2_data/lipidkey.csv', sep = '\t')
key = key[!duplicated(key$Key),]
rownames(key) = key$Key
proteomics = read.csv('../processed_data/for_plotting/ABCA7_proteomics.csv')
wgs = read.csv('../raw_data/metadata/WGS_sample_QC_info.csv')

In [31]:
head(wgs)

,projid,WGS_id,Source.Tissue.Type,Gender_match,Concordant_with_array,Concordant_with_RNAseq,Duplicate,QC,GQN
,<int>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<dbl>
1,68539908,SM-CTEGU,Brain-Cerebellum,FALSE,NA,FALSE,identical to SM-CTELW,Fail (discordant with RNA-seq;gender mismatch),8.8
2,20882194,SM-CJEI2,Whole Blood,TRUE,NA,FALSE,identical to SM-CTEFA,Fail (discordant with RNA-seq),8.2
3,50101523,SM-CTEDY,Whole Blood,FALSE,FALSE,NA,NA,Fail (discordant with SNP array;gender mismatch),7.3
4,50410283,SM-CTEGE,Brain-Cerebellum,FALSE,FALSE,NA,identical to SM-CTEN4,Fail (discordant with SNP array;gender mismatch),9.3
5,75175129,SM-CJFKP,Brain-DLPFC,FALSE,FALSE,NA,identical to SM-CTDVH,Fail (discordant with SNP array;gender mismatch),4.4
6,10100736,SM-CTDVR,Brain-Cerebellum,TRUE,FALSE,NA,identical to SM-CJFO9,Fail (discordant with SNP array),8.9


In [56]:
data = merge(proteomics[proteomics$LOF==0,], wgs[,c('projid', 'WGS_id')], by='projid')

In [35]:
# load Henne Holstege et al data
df = read.csv('../processed_data/holstege_et_al//41588_2022_1208_MOESM4_ESM.csv', skip = 2, header = TRUE)

# merge with ABCA7 variant info
variant_info = read.csv('../raw_data/ROSMAP_WGS/HIGHandMED_coding_annotations_syn11724057_subset.csv', check.names=FALSE)
variant_info$Variant = paste0(variant_info$CHROM,':',variant_info$POS,':',variant_info$REF_x, '>', variant_info$ALT_0)
df = df[,c('Variant', 'REVEL', 'LOF')]
var_info = merge(variant_info, df, by = 'Variant')
var_info = var_info[var_info$GENE=='ABCA7',]

genotypes = var_info[,unlist(lapply(names(var_info), function(x) sum(startsWith(x, c('ROS', 'MAP', 'SM')))))>0]
genotypes[genotypes=='0/0']=0
genotypes[genotypes=='0/1']=1
genotypes[genotypes=='1/0']=1
genotypes[genotypes=='1/1']=2
df = apply(genotypes,2, function(x){as.numeric(x)})

variant_categories = as.data.frame(colSums(df[var_info$LOF.y==1,]))
temp = as.numeric(unlist(genotypes[var_info$ID=='rs3752246',]))                                  
temp = -1*(temp-2)
variant_categories$rs3752246 = temp

# Create a pie chart
percentage_col1_1 <- mean(variant_categories$Lof == 1) * 100

pdf('../pdf_figures/percentage_lof.pdf')
ggplot(variant_categories, aes(x = "",fill = factor(variant_categories[,'Lof']), color=factor(variant_categories[,'Lof']))) +
  geom_bar(width = 1, stat = "count") +
  coord_polar(theta = "y") +   scale_color_manual(values = c("white", "white")) +  # Set black outline

  scale_fill_manual(values = c("grey", "red"), name='Lof') +   geom_text(x = 1, y = 0, label = paste0(round(percentage_col1_1, 1), "%"), color = "white", size = 8) + theme_classic() +   theme_void() +   theme(legend.text=element_text(size=15)) 

dev.off()

# Create a pie chart

pdf('../pdf_figures/percentage_common.pdf')
percentage_col1_1 <- mean(variant_categories$common == 1) * 100

ggplot(variant_categories, aes(x = "",fill = factor(variant_categories[,'ABCA7 p.1527']), color=factor(variant_categories[,'ABCA7 p.1527']))) +
  geom_bar(width = 1, stat = "count") +
  coord_polar(theta = "y") +   scale_color_manual(values = c("white", "white")) +  # Set black outline

  scale_fill_manual(values = c("grey", "salmon"), name='ABCA7 p.1527') +   geom_text(x = 1, y = 0, label = paste0(round(percentage_col1_1, 1), "%"), color = "white", size = 8) + theme_classic() +   theme_void() +   theme(legend.text=element_text(size=15)) 

dev.off()

Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
